In [1]:
import soundfile # to read audio file
import numpy as np
import librosa # to extract speech features
import glob
import os
import pandas as pd
import os
import csv # Preprocessing
import pickle # to save model after training
from sklearn.model_selection import train_test_split # for splitting training and testing
from sklearn.neural_network import MLPClassifier # multi-layer perceptron model
from sklearn.metrics import accuracy_score # to measure how good we are

In [2]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [3]:
# Saving the datset into a CSV file

header = 'filename chroma_stft melspectogram spectral_contrast tonnetz'
for i in range(1, 41):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

file = open('emotiondataset.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
emotions = 'Angry Disgusted Fearful Happy Neutral Sad Surprised'.split()

for m in emotions:
    for filename in os.listdir(f'Emotions/{m}'):
        audioname = f'Emotions/{m}/{filename}'
        y, sr = librosa.load(audioname, mono=True, duration=30)
        stft = np.abs(librosa.stft(y))
        chroma_stft = librosa.feature.chroma_stft(y=stft, sr=sr)
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr)
        spec_cont = librosa.feature.spectral_contrast(y=stft, sr=sr)
        tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(mel_spec)} {np.mean(spec_cont)} {np.mean(tonnetz)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {m}'
        file = open('emotiondataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=112
  return f(*args, **kwargs)
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=896
  return f(*args, **kwargs)
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=105
  return f(*args, **kwargs)
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=838
  return f(*args, **kwargs)
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=159
  return f(*args, **kwargs)
c:\Us

In [12]:
# all emotions on Audio Emotions dataset
int2emotion = {
    "01": "Neutral",
    "02": "Happy",
    "03": "Sad",
    "04": "Angry",
    "05": "Fearful",
    "06": "Disgusted",
    "07": "Surprised"
}

# we allow only these emotions
AVAILABLE_EMOTIONS = {
    "Neutral",
    "Angry",
    "Happy",
    "Sad",
    "Fearful",
    "Disgusted",
    "Surprised"
}

def load_data(test_size=0.2):
    X, y = [], []
    
    for m in emotions:
        for file in glob.glob(f'Emotions/{m}/*.wav'):
            # get the base name of the audio file
            basename = os.path.basename(file)
            # get the emotion label
            emotion = int2emotion[basename.split("-")[2]]
            # we allow only AVAILABLE_EMOTIONS we set
            if emotion not in AVAILABLE_EMOTIONS:
                continue
            # extract speech features
            features = extract_feature(file, mfcc=True, chroma=True, mel=True)
            # add to data
            X.append(features)
            y.append(emotion)
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size, random_state=7)

In [5]:
# load Audio emotion dataset, 75% training 25% testing
X_train, X_test, y_train, y_test = load_data(test_size=0.25)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_29904\1163463125.py:31: FutureWarning: Pass y=[0.0017395  0.0017395  0.00170898 ... 0.0015564  0.00152588 0.00152588] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_29904\1163463125.py:31: FutureWarning: Pass y=[0.00152588 0.00149536 0.00149536 ... 0.00140381 0.00143433 0.00146484] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_29904\1163463125.py:31: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_299

In [6]:
# print some details
# number of samples in training data
print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])
# number of features used
# this is a vector of features extracted 
# using extract_features() function
print("[+] Number of features:", X_train.shape[1])

[+] Number of training samples: 22
[+] Number of testing samples: 8
[+] Number of features: 180


In [7]:
# best model, determined by a grid search
model_params = {
    'alpha': 0.01,
    'batch_size': 256,
    'epsilon': 1e-08, 
    'hidden_layer_sizes': (300,), 
    'learning_rate': 'adaptive', 
    'max_iter': 500, 
}

In [8]:
# initialize Multi Layer Perceptron classifier
# with best parameters ( so far )
model = MLPClassifier(**model_params)

In [9]:
# train the model
print("[*] Training the model...")
model.fit(X_train, y_train)

[*] Training the model...


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:621: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(


MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [10]:
# predict 25% of data to measure how good we are
y_pred = model.predict(X_test)

# calculate the accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 62.50%


In [11]:
# now we save the model
# make result directory if doesn't exist yet
if not os.path.isdir("result"):
    os.mkdir("result")

pickle.dump(model, open("result/mlp_classifier.model", "wb"))